<div style="background: linear-gradient(135deg, #2c3e50 0%, #34495e 100%); padding: 20px; border-radius: 10px; color: white; text-align: center; margin-bottom: 20px;">
    <h1 style="margin: 0; font-size: 2.5em; font-weight: 700; text-shadow: 2px 2px 4px rgba(0,0,0,0.3);">
        🔍 Badanie ograniczeń dbzero
    </h1>
    <p style="margin: 10px 0 0 0; font-size: 1.2em; opacity: 0.9;">
        <strong>Część 3/12:</strong> Sprawdzamy granice możliwości pamięci
    </p>
</div>

<div style="background-color: #ecf0f1; border-left: 4px solid #34495e; padding: 15px; margin: 15px 0; border-radius: 5px;">
    <p style="margin: 0; font-size: 1.1em; color: #2c3e50;">
        💡 <strong>W tej części:</strong> Poznamy jak dbzero radzi sobie z ograniczeniami pamięci i dlaczego nigdy więcej nie będziesz się martwić o błędy "out of memory".
    </p>
</div>

In [1]:
import dbzero as db0
from mem_charts import mem_usage_chart, random_string
from bokeh.io import show, output_notebook
import concurrent.futures

<div style="background-color: #f8f9fa; border: 1px solid #95a5a6; border-radius: 6px; padding: 15px; margin: 15px 0;">
    <p style="margin: 0; display: flex; align-items: flex-start; gap: 10px; color: #2c3e50; font-size: 1.05em; line-height: 1.5;">
        <span style="font-size: 1.5em;">📊</span>
        <span><strong>Wizualizacja:&nbsp;</strong> Zaimportowaliśmy pakiet 'bokeh' do wizualizacji obecnego wykorzystania pamięci na wykresie.</span>
    </p>
</div>

<div style="background-color: #ecf0f1; border: 1px solid #bdc3c7; border-radius: 6px; padding: 15px; margin: 15px 0;">
    <p style="margin: 0; display: flex; align-items: center; gap: 10px; color: #2c3e50; font-size: 1.1em;">
        <span style="font-size: 1.5em;">🔧</span>
        <strong>Setup:&nbsp;</strong> Stwórzmy również executor, aby móc uruchamiać zadania Pythona w tle (w oddzielnym wątku).
    </p>
</div>

In [2]:
executor = concurrent.futures.ThreadPoolExecutor(max_workers=1)

<div style="background: linear-gradient(90deg, #34495e 0%, #2c3e50 100%); padding: 15px; border-radius: 8px; color: white; margin: 20px 0;">
    <h3 style="margin: 0; font-size: 1.8em; display: flex; align-items: center;">
        📈 Monitor pamięci w czasie rzeczywistym
    </h3>
    <p style="margin: 10px 0 0 0; font-size: 1.05em; opacity: 0.95;">
        Wykres poniżej przedstawia żywe wykorzystanie pamięci przez obecny proces, odświeżane co 1 sekundę.
    </p>
</div>

In [3]:
output_notebook(resources=None, verbose=False, hide_banner=True)
show(mem_usage_chart, notebook_url="http://127.0.0.1:8888", port=8889)

<div style="background-color: #f8f9fa; border: 2px solid #e74c3c; border-radius: 8px; padding: 20px; margin: 15px 0;">
    <div style="display: flex; align-items: flex-start; gap: 15px;">
        <div style="font-size: 2em; color: #e74c3c;">⚠️</div>
        <div>
            <h4 style="margin: 0 0 10px 0; color: #c0392b; font-size: 1.3em;">Test standardowego Pythona - Uwaga na pamięć!</h4>
            <p style="margin: 0; font-size: 1.1em; line-height: 1.6; color: #2c3e50;">
                Zobaczmy najpierw, jak rośnie wykorzystanie pamięci podczas uruchamiania zwykłego kodu Pythona. Dodajemy <strong style="color: #e74c3c;">50 000 losowych elementów tekstowych</strong> do zwykłej listy Pythona w <strong style="color: #e74c3c;">100 iteracjach</strong>.
            </p>
            <p style="margin: 10px 0 0 0; font-size: 1.05em; color: #7f8c8d; font-style: italic;">
                To łącznie <strong>5 milionów dodanych elementów danych</strong>. Zobaczmy, jak to działa...
            </p>
        </div>
    </div>
</div>

In [4]:
result = []
def generate_sequence(result, length, batch):
    for _ in range(length):
        result.extend([random_string() for _ in range(batch)])
    print("Task finished")

In [5]:
task = executor.submit(generate_sequence, result, length=100, batch = 50000)

Task finished
Task finished


<div style="background: linear-gradient(135deg, #e74c3c 0%, #c0392b 100%); padding: 20px; border-radius: 10px; color: white; margin: 20px 0;">
    <div style="display: flex; align-items: center; gap: 15px;">
        <div style="font-size: 2.5em;">💥</div>
        <div>
            <h4 style="margin: 0 0 10px 0; font-size: 1.4em; font-weight: bold;">Typowy problem Pythona</h4>
            <p style="margin: 0; font-size: 1.05em; line-height: 1.5; opacity: 0.95;">
                Pamięć po prostu nadal rośnie i rośnie i ostatecznie zakończyłaby proces błędem <strong>"brak pamięci"</strong>. W Pythonie możemy zwolnić pamięć poprzez zwykłe opróżnienie listy.
            </p>
        </div>
    </div>
</div>

In [6]:
result = []

<div style="background: linear-gradient(90deg, #27ae60 0%, #2ecc71 100%); padding: 20px; border-radius: 10px; color: white; margin: 20px 0;">
    <h3 style="margin: 0 0 15px 0; font-size: 1.8em; display: flex; align-items: center; gap: 10px;">
        🚀 Dobrze, więc jak dbzero różni się w tej kwestii?
    </h3>
    <p style="margin: 0; font-size: 1.1em; line-height: 1.6; opacity: 0.95;">
        W <strong>dbzero</strong> pracujesz przez większość czasu jak ze zwykłym kodem Pythona, ale nie musisz już martwić się o ograniczenia pamięci. To prawda, nawet jeśli masz do czynienia z <strong>terabajtami danych</strong>, Twój proces nigdy nie przekroczy limitów, które sam zdefiniujesz.
    </p>
</div>

<div style="background-color: #ecf0f1; border-left: 4px solid #3498db; padding: 15px; margin: 15px 0; border-radius: 5px;">
    <p style="margin: 0; font-size: 1.1em; color: #2c3e50; display: flex; align-items: center; gap: 10px;">
        <span style="font-size: 1.5em; color: #3498db;">🔧</span>
        <strong>Inicjalizacja:&nbsp;</strong> Po inicjalizacji&nbsp;<strong>dbzero</strong>&nbsp;zajmie niewielką ilość Twojej pamięci...
    </p>
</div>

In [7]:
db0.init(dbzero_root = "/dbzero", prefix = "data")

<div style="background-color: #f8f9fa; border: 1px solid #95a5a6; border-radius: 6px; padding: 15px; margin: 15px 0;">
    <p style="margin: 0; display: flex; align-items: flex-start; gap: 10px; color: #2c3e50; font-size: 1.05em; line-height: 1.5;">
        <span style="font-size: 1.5em;">⚙️</span>
        <span><strong>Kontrola pamięci:&nbsp;</strong> Ale możesz kontrolować, ile dodatkowej pamięci używa, wywołując metodę <code style="background-color: #ecf0f1; padding: 2px 6px; border-radius: 4px; color: #7f8c8d; font-weight: bold;">set_cache_size</code>.</span>
    </p>
</div>

In [8]:
db0.set_cache_size(128 << 20)

<div style="background: linear-gradient(135deg, #3498db 0%, #2980b9 100%); padding: 20px; border-radius: 10px; color: white; margin: 20px 0;">
    <div style="display: flex; align-items: center; gap: 15px;">
        <div style="font-size: 2.5em;">🎯</div>
        <div>
            <h4 style="margin: 0 0 10px 0; font-size: 1.4em; font-weight: bold;">Test dbzero - Obserwuj magię!</h4>
            <p style="margin: 0; font-size: 1.05em; line-height: 1.5; opacity: 0.95;">
                Powtórzmy teraz test używając <code style="background-color: rgba(255,255,255,0.2); padding: 2px 6px; border-radius: 4px; font-weight: bold;">db0.list</code> (obiekt listy w przestrzeni dbzero). Obserwuj uważnie, jak wykorzystanie pamięci <strong>zatrzymuje się w pewnym momencie</strong> (gdy zostanie osiągnięty zdefiniowany limit cache) i nie rośnie bez względu na to, ile danych umieścisz w swojej liście.
            </p>
        </div>
    </div>
</div>

In [9]:
db0_result = db0.list()

In [10]:
def db0_generate_sequence(result, length, batch):
    for _ in range(length):
        result.extend([random_string() for _ in range(batch)]) 
    print("Task finished")

In [11]:
task = executor.submit(db0_generate_sequence, db0_result, length=100, batch = 50000)

In [12]:
print(len(db0_result))
db0_result[12313]

5000000


'oeqVRJbN24Un'

<div style="background: linear-gradient(135deg, #9b59b6 0%, #8e44ad 100%); padding: 25px; border-radius: 12px; color: white; margin: 20px 0;">
    <h3 style="margin: 0 0 15px 0; font-size: 1.8em; display: flex; align-items: center; gap: 10px;">
        🤔 No więc gdzie w rzeczywistości są przechowywane dane w tym przypadku?
    </h3>
    <div style="background-color: rgba(255,255,255,0.1); border-radius: 8px; padding: 20px; border: 1px solid rgba(255,255,255,0.2); margin-top: 15px;">
        <p style="margin: 0; font-size: 1.05em; line-height: 1.6; opacity: 0.95;">
            <strong>dbzero</strong> implementuje algorytmy wymiany pamięci. Pobiera dane z chmury lub, w przypadku wersji lokalnej, z systemu plików na zasadzie <strong>"według potrzeby"</strong> i przechowuje je w lokalnym cache, aby umożliwić szybki dostęp w przyszłości.
        </p>
        <p style="margin: 15px 0 0 0; font-size: 1.05em; opacity: 0.9; font-style: italic;">
            💡 Proces jest całkowicie przezroczysty dla programisty.
        </p>
    </div>
</div>

In [ ]:
db0.close()

<div style="background: linear-gradient(135deg, #2c3e50 0%, #34495e 100%); padding: 25px; border-radius: 12px; color: white; text-align: center; margin: 30px 0;">
    <h2 style="margin: 0 0 15px 0; font-size: 1.8em; font-weight: 700;">
        Świetna robota! 
    </h2>
    <p style="margin: 0 0 15px 0; font-size: 1.2em; opacity: 0.9;">
        Ukończyłeś trzecią część kursu dbzero
    </p>
    <div style="display: flex; justify-content: center; align-items: center; gap: 20px; margin-top: 20px; flex-wrap: wrap;">
        <div style="background-color: rgba(255,255,255,0.15); padding: 10px 20px; border-radius: 20px; font-size: 0.95em; border: 1px solid rgba(255,255,255,0.2);">
            Część 3/12 ukończona
        </div>
        <div style="background-color: rgba(255,255,255,0.15); padding: 10px 20px; border-radius: 20px; font-size: 0.95em; border: 1px solid rgba(255,255,255,0.2);">
            Następna: Część 4
        </div>
    </div>
    <p style="margin: 20px 0 0 0; font-size: 0.95em; opacity: 0.8; font-style: italic;">
        💡 Teraz wiesz jak dbzero radzi sobie z ograniczeniami pamięci!
    </p>
</div>